# RUN SfM Pipeline with Deep-Image-Matching


In [2]:
%reload_ext autoreload
%autoreload 2

from pprint import pprint

import deep_image_matching as dim

logger = dim.logger

Deep Image Matching loaded in 10.615 seconds.


## Initialization

Get the list of possible pipelines and matching strategy and chose one of them.


In [3]:
print("Available pipelines:")
pprint(dim.Config.get_pipelines())
print("Available matching strategy:")
pprint(dim.Config.get_matching_strategies())

Available pipelines:
['superpoint+lightglue',
 'superpoint+superglue',
 'superpoint+kornia_matcher',
 'disk+lightglue',
 'aliked+lightglue',
 'orb+kornia_matcher',
 'sift+kornia_matcher',
 'loftr',
 'se2loftr',
 'roma',
 'keynetaffnethardnet+kornia_matcher',
 'dedode+kornia_matcher',
 'xfeat+kornia_matcher',
 'sift+lightglue',
 'dedode+lightglue',
 'sift_pycolmap+lightglue']
Available matching strategy:
['bruteforce',
 'sequential',
 'retrieval',
 'custom_pairs',
 'matching_lowres',
 'covisibility']


Now you have to build a dictionary with the input processing parameters (they are the same as the input parameters for the CLI) and pass it to the Config class to initialize the configuration object. Refer to the [documentation](https://3dom-fbk.github.io/deep-image-matching/) for more information about the parameters.

Note that the `dir` defines the project directory, where the images are stored and the results will be saved.
Deep-Image-Matching will search for the images inside an 'image' subdirectory and will save the results in a 'results\_{processing_params}' subdirectory, where {processing_params} are some information on the processing parameters used.

By default DIM will not run if the output directory already exists, to avoid overwriting previous results. If you want to overwrite the results, you can set the `force` parameter to True. We have not implemented the possibility to recover the previous results yet (e.g., by using existing extracted features), but we may add it in the future.

The `config_file` parameter is the path to the configuration file (optional). In this file you can specify all the parameters that you need for controlling the feature extraction and matching. Refer to the [documentation](https://3dom-fbk.github.io/deep-image-matching/advanced_configuration/) for more information about how to write this file. Note that this file il optional, if you don't pass it, the default parameters will be used.

If you use set `verbose` to True, DIM will log all the processing steps and it will save some figures with the extracted features and the matches in a `debug` folder inside the results directory. Note that this will slow down the processing and will create a lot of files if the dataset is big. It is reccomended to use it only for testing or debugging purposes.


In [4]:
params = {
    "dir": "./assets/example_cyprus",
    "pipeline": "superpoint+lightglue",
    "config_file": "./assets/example_cyprus/config_superpoint+lightglue.yaml",
    "strategy": "matching_lowres",
    "quality": "high",
    "tiling": "preselection",
    "skip_reconstruction": False,
    "force": True,
    "camera_options": "./assets/example_cyprus/cameras.yaml",
    "openmvg": None,
    "verbose": False,
}
config = dim.Config(params)

# Save the configuration to a json file for later use
config.save()

2024-05-30 09:50:08 | [WARNING ] assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_high already exists, but the '--force' option is used. Deleting the folder.
Using a custom configuration file: /home/francesco/phd/deep-image-matching/assets/example_cyprus/config_superpoint+lightglue.yaml


You can check the configuration object.


In [5]:
print("Config general:")
pprint(config.general)

Config general:
{'camera_options': './assets/example_cyprus/cameras.yaml',
 'config_file': None,
 'db_path': None,
 'dir': None,
 'force': True,
 'geom_verification': <GeometricVerification.PYDEGENSAC: 1>,
 'global_feature': None,
 'graph': True,
 'gui': False,
 'gv_confidence': 0.99999,
 'gv_threshold': 4,
 'image_dir': PosixPath('assets/example_cyprus/images'),
 'images': None,
 'matching_strategy': 'matching_lowres',
 'min_inlier_ratio_per_pair': 0.25,
 'min_inliers_per_pair': 10,
 'min_matches_per_tile': 10,
 'openmvg_conf': None,
 'output_dir': PosixPath('assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_high'),
 'outs': None,
 'overlap': None,
 'pair_file': PosixPath('assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_high/pairs.txt'),
 'pipeline': None,
 'preselection_pipeline': 'superpoint+lightglue',
 'quality': <Quality.HIGH: 3>,
 'retrieval': None,
 'skip_reconstruction': False,
 'strategy': 'matching_lowres',
 'tile_overlap':

In [6]:
print("Config extractor:")
pprint(config.extractor)

Config extractor:
{'fix_sampling': False,
 'keypoint_threshold': 0.005,
 'max_keypoints': 8000,
 'name': 'superpoint',
 'nms_radius': 4,
 'remove_borders': 4}


In [7]:
print("Config matcher:")
pprint(config.matcher)

Config matcher:
{'depth_confidence': 0.95,
 'filter_threshold': 0.1,
 'flash': True,
 'mp': False,
 'n_layers': 9,
 'name': 'lightglue',
 'width_confidence': 0.99}


If you want to change some parameters, you can do it by accessing the dictionary and changing the values. For example, if you want to change the number of features to extract, you can do it like this:

```python
config.extractor['max_keypoints'] = 1000
```

Note that different Extractors and Matchers have different parameters, so be careful when changing them.
You can check the parameters of the Extractor and Matcher by accessing the corresponding dictionary, such as:

```python
print(dim.extractors.SuperPointExtractor.get_default_conf())
print(dim.matchers.LightGlueMatcher.get_default_conf())
```


## Run the extraction and matching

First, you have to create an instance of the ImageMatching class and pass the configuration object to it.


In [8]:
matcher = dim.ImageMatcher(config)

Loaded SuperPoint model
Loaded SuperPoint model
Loaded LightGlue model
2024-05-30 09:50:10 | [INFO    ] Running image matching with the following configuration:
2024-05-30 09:50:10 | [INFO    ]   Image folder: assets/example_cyprus/images
2024-05-30 09:50:10 | [INFO    ]   Output folder: assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_high
2024-05-30 09:50:10 | [INFO    ]   Number of images: 10
2024-05-30 09:50:10 | [INFO    ]   Matching strategy: matching_lowres
2024-05-30 09:50:10 | [INFO    ]   Image quality: HIGH
2024-05-30 09:50:10 | [INFO    ]   Tile selection: PRESELECTION
2024-05-30 09:50:10 | [INFO    ]   Feature extraction method: superpoint
2024-05-30 09:50:10 | [INFO    ]   Matching method: lightglue
2024-05-30 09:50:10 | [INFO    ]   Geometric verification: PYDEGENSAC
2024-05-30 09:50:10 | [INFO    ]   CUDA available: True


### Run Full Pipeline

Then you can run the full pipeline (extraction and matching) by calling the `run` method.
This method will automatically run all the steps needed to extract the features and match the images. It will return the path to the h5 files containing the features and the matches.

The `features.h5` file contains the features extracted from each images, while the `matches.h5` file contains the indices of the features matched.

You can decide to automatically save the results in a COLMAP database by setting the `export_to_colmap` parameter to True. The database will be saved in the results directory with the name `database.db`.
If you set `verbose` to True (default is False), it will also save figures in the `debug` folder and print (a lot) more information on the process. Note that it may create quite a lot of files if the dataset is big, so it is recommended to use it only for testing or debugging purposes.


In [9]:
# Run image matching
feature_path, match_path = matcher.run(export_to_colmap=False, verbose=False)

Loaded SuperPoint model
2024-05-30 09:50:10 | [INFO    ] Extracting features from downsampled images...


  0%|          | 0/10 [00:00<?, ?it/s]/home/francesco/miniforge3/envs/dim/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 10/10 [00:02<00:00,  4.23it/s]

2024-05-30 09:50:12 | [INFO    ] Matching downsampled images...



100%|██████████| 45/45 [00:00<00:00, 51.82it/s]

2024-05-30 09:50:13 | [INFO    ] Extracting features with superpoint...
2024-05-30 09:50:13 | [INFO    ] superpoint configuration: 


{'fix_sampling': False,
 'keypoint_threshold': 0.005,
 'max_keypoints': 8000,
 'name': 'superpoint',
 'nms_radius': 4,
 'remove_borders': 4}


100%|██████████| 10/10 [00:02<00:00,  3.43it/s]

2024-05-30 09:50:16 | [INFO    ] Features extracted!
2024-05-30 09:50:16 | [INFO    ] Matching features with lightglue...
2024-05-30 09:50:16 | [INFO    ] lightglue configuration: 
{'add_laf': False,
 'add_scale_ori': False,
 'depth_confidence': 0.95,
 'descriptor_dim': 256,
 'filter_threshold': 0.1,
 'flash': True,
 'input_dim': 256,
 'mp': False,
 'n_layers': 9,
 'name': 'lightglue',
 'num_heads': 4,
 'scale_coef': 1.0,
 'weights': None,
 'width_confidence': 0.99}
2024-05-30 09:50:16 | [INFO    ] Matching features...
2024-05-30 09:50:16 | [INFO    ] 



100%|██████████| 28/28 [00:13<00:00,  2.06it/s]

2024-05-30 09:50:30 | [INFO    ] [Timer] | [matching] generate_pairs=4.992, extract_features=2.941, Match pair=0.614, Total execution=21.583
2024-05-30 09:50:30 | [INFO    ] [Timer] | [Deep Image Matching] Total execution=0.000


### Run Step-by-Step

Alternatevely, you can run the process step-by-step to have more control on the process.

First, generate pairs of images to be matched. This will create a file with the names of the imaes that will be matched together.
The pairs generation is carried out based on the `strategy` parameter in the configuration object.


In [13]:
pair_path = matcher.generate_pairs()

Loaded SuperPoint model
2024-05-30 09:55:09 | [INFO    ] Extracting features from downsampled images...


100%|██████████| 10/10 [00:00<00:00, 17.54it/s]

2024-05-30 09:55:10 | [INFO    ] Matching downsampled images...



100%|██████████| 45/45 [00:00<00:00, 50.60it/s]


If your dataset has images with >40/50 deg rotations, it is suggested to rotate the images upright before doing the extraction and matching, which is particularly useful for deep-learning approaches that usually are not rotation invariant (e.g. SuperPoint).
To rotate images so they will be all "upright":


In [14]:
matcher.rotate_upright_images()

2024-05-30 09:55:48 | [INFO    ] Rotating images upright...


TypeError: Invalid config object. 'custom_config' must be a Config object

Run the feature extraction on the images using the chosen extraction method. This will extract the features from the images and save them in the `features.h5` file.


In [ ]:
feature_path = matcher.extract_features()

Run the matching on the extracted features. This will match the features and save the matches in the `matches.h5` file.


In [ ]:
match_path = matcher.match_pairs(feature_path)

If features have been extracted on "upright" images, this function bring features back to their original image orientation

In [ ]:
matcher.rotate_back_features(feature_path)

### Access the results from the h5 files


You can access the feautes and the matches from the h5 files using the h5py library. Here is an example on how to read the features and the matches from the h5 files:


In [11]:
# The `features.h5` file

import h5py

images = matcher.image_list

with h5py.File(feature_path, "r") as f:
    print(f.keys())
    print(f[images[0].name].keys())
    print(f[images[0].name]["keypoints"][:])
    print(f[images[1].name].keys())
    print(f[images[1].name]["keypoints"][:])

<KeysViewHDF5 ['DSC_6466.JPG', 'DSC_6467.JPG', 'DSC_6468.JPG', 'DSC_6469.JPG', 'DSC_6470.JPG', 'DSC_6471.JPG', 'DSC_6472.JPG', 'DSC_6473.JPG', 'DSC_6474.JPG', 'DSC_6475.JPG']>
<KeysViewHDF5 ['descriptors', 'image_size', 'keypoints', 'scores', 'tile_idx']>
[[   2.  134.]
 [   2.  458.]
 [   5.  624.]
 ...
 [1492.  443.]
 [1492.  448.]
 [1493.  829.]]
<KeysViewHDF5 ['descriptors', 'image_size', 'keypoints', 'scores', 'tile_idx']>
[[   2.  894.]
 [   3.  867.]
 [   6.  273.]
 ...
 [1490.  749.]
 [1491.  436.]
 [1497.  147.]]


In [12]:
# The `matches.h5` file

with h5py.File(match_path, "r") as f:
    print(f.keys())
    g0 = f[images[0].name]
    print(g0.keys())
    g1 = g0[images[1].name]
    print(g1.__array__())

<KeysViewHDF5 ['DSC_6466.JPG', 'DSC_6467.JPG', 'DSC_6469.JPG', 'DSC_6470.JPG', 'DSC_6472.JPG', 'DSC_6473.JPG', 'DSC_6474.JPG']>
<KeysViewHDF5 ['DSC_6467.JPG', 'DSC_6469.JPG', 'DSC_6470.JPG', 'DSC_6472.JPG', 'DSC_6473.JPG', 'DSC_6474.JPG', 'DSC_6475.JPG']>
[[1007 3522]
 [1467 3787]
 [1679 2263]
 ...
 [4956 1903]
 [4957 1960]
 [4958 1832]]


## Export in colmap format

Then you can use the `export_to_colmap` function that will read the features and the matches from the h5 files and will save them in a COLMAP sqlite database.
DIM assigns camera models to images based on the options defined in `cameras.yaml` file.


In [10]:
database_path = config.general["output_dir"] / "database.db"
dim.io.export_to_colmap(
    img_dir=config.general["image_dir"],
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_config_path=config.general["camera_options"],
)

2024-05-30 09:50:30 | [WARNING ] Was not possible to load the first image to initialize cam0
2024-05-30 09:50:30 | [WARNING ] Was not possible to load the first image to initialize cam1


100%|██████████| 10/10 [00:00<00:00, 906.45it/s]
28it [00:00, 6911.92it/s]             
28it [00:00, 7246.28it/s]             


Alternatively you can assign camera models with a dictionary and pass the parameter `camera_config_dict` to the `export_to_colmap` function. This dictionary will override the camera models assigned in the `cameras.yaml` file.

For images not assigned to specific `cam<x>` camera groups, the options specified under `general` are applied. The `camera_model` can be selected from `["simple-pinhole", "pinhole", "simple-radial", "opencv"]`. It's worth noting that it's easily possible to extend this to include all the classical COLMAP camera models. Cameras can either be shared among all images (`single_camera == True`), or each camera can have a different camera model (`single_camera == False`).

A subset of images can share intrinsics using `cam<x>` key, by specifying the `camera_model` along with the names of the images separated by commas. There's no limit to the number of `cam<x>` entries you can use.

**Note**: Use the SIMPLE-PINHOLE camera model if you want to export the solution to Metashape later, as there are some bugs in COLMAP (or pycolamp) when exportingthe solution in the Bundler format.
e.g., using FULL-OPENCV camera model, the principal point is not exported correctly and the tie points are wrong in Metashape.

```python
camera_config_dict = {
    "general": {
        "camera_model": "pinhole",  # ["simple-pinhole", "pinhole", "simple-radial", "opencv"]
        "single_camera": True,
    },
    "cam0": {
        "camera_model": "pinhole",
        "images": "DSC_6468.JPG,DSC_6468.JPG",
    },
    "cam1": {
        "camera_model": "pinhole",
        "images": "",
    },
}
```


## Run reconstruction

You can run the reconstruction with pycolmap, OpenMVG or MICMAC. The suggested method is pycolmap, as it is the most integrated with DIM.

Note that pycolmap is not installed by default with DIM, so you have to install it manually. You can find the installation instructions [here](https://3dom-fbk.github.io/deep-image-matching/installation/).


In [ ]:
try:
    import pycolmap
except ImportError:
    logger.error("Pycomlap is not available.")

If the pycolmap module is imported, you can define all the parameters for the COLAMP reconstruction.
You can check all the available parameters with:

```python
print(pycolmap.IncrementalPipelineOptions().summary())
```

Alternatevely, you can leave the dictionary empty for using the default confiuration

```python
reconst_opts = {}
```


In [ ]:
# Run reconstruction
opt = dict(
    triangulation=dict(
        ignore_two_view_tracks=False,
        min_angle=0.5,
    ),
    mapper=dict(filter_min_tri_angle=0.5, filter_max_reproj_error=5.0),
)
refine_intrinsics = False
verbose = False

model = dim.reconstruction.pycolmap_reconstruction(
    database_path=config.general["output_dir"] / "database.db",
    sfm_dir=config.general["output_dir"],
    image_dir=config.general["image_dir"],
    refine_intrinsics=refine_intrinsics,
    options=opt,
    verbose=verbose,
)

In [ ]:
# Print COLMAP camera values
print(list(model.cameras.values()))